# Utility

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import backend as K

Using TensorFlow backend.


In [2]:
class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename)
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['sentence'], float):
                texts.append(str(row['sentence']))
            else:
                texts.append(row['sentence'])

            labels.append(row['label'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = Tokenizer(num_words=10000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_metric(self, y_true, y_pred):
        accuracyScore = accuracy_score(y_true, y_pred)

        # binary: Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.
        precisionScoreBinary = precision_score(y_true, y_pred, average='binary')
        recallScoreBinary = recall_score(y_true, y_pred, average='binary')
        f1ScoreBinary = f1_score(y_true, y_pred, average='binary')

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def print_metric(self, accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary):
        print("Accuracy: " + str(accuracyScore))
        print("Precision: " + str(precisionScoreBinary))
        print("Recall: " + str(recallScoreBinary))
        print("F1-Score: " + str(f1ScoreBinary))
        print(str(accuracyScore) + "," + str(precisionScoreBinary) + "," + str(recallScoreBinary) + "," + str(
            f1ScoreBinary))

    def get_testing_metric(self, y_test, y_pred):
        # metric for Testing Data
        # print("Testing Data")
        accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = self.get_metric(y_test, y_pred)
        # print()

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_trial_data(self, textsTraining, labelsTraining, textsTrial, labelsTrial, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining), np.asarray(textsTrial)
        y_train, y_val = np.asarray(labelsTraining), np.asarray(labelsTrial)

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)
    
    def recall_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(self, y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))

# Finite State Machine

In [3]:
import random

def FSM():
    fsm = {}
    fsm[0] = {'src': 0, 'dst': 1, 'layer': 'embedding_layer', 'next_path': [1]}
    fsm[1] = {'src': 1, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[2] = {'src': 2, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[3] = {'src': 2, 'dst': 3, 'layer': 'maxpooling_layer', 'next_path': [4]}
    fsm[4] = {'src': 3, 'dst': 2, 'layer': 'convolutional_layer', 'next_path': [2, 3, 5]}
    fsm[5] = {'src': 2, 'dst': 4, 'layer': 'global_maxpooling_layer', 'next_path': [6, 7]}
    fsm[6] = {'src': 4, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[7] = {'src': 4, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [10, 12]}
    fsm[8] = {'src': 5, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[9] = {'src': 5, 'dst': 6, 'layer': 'dropout_layer', 'next_path': [10, 12]}
    fsm[10] = {'src': 6, 'dst': 5, 'layer': 'dense_layer', 'next_path': [8, 9, 11]}
    fsm[11] = {'src': 5, 'dst': 7, 'layer': 'output_layer', 'next_path': []}
    fsm[12] = {'src': 6, 'dst': 7, 'layer': 'output_layer', 'next_path': []}

    return fsm

def mutateFSM():
    fsm = {}
    fsm['convolutional_layer'] = {'before': ['convolutional_layer'],
                                  'after': ['convolutional_layer'], 'change': 'maxpooling_layer'}
    fsm['maxpooling_layer'] = {'before': ['convolutional_layer'],
                               'after': ['convolutional_layer'], 'change': 'convolutional_layer'}
    fsm['dense_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                          'after': ['dense_layer'], 'change': 'dropout_layer'}
    fsm['dropout_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                            'after': ['dense_layer'], 'change': 'dense_layer'}

    return fsm

def addFSM():
    fsm = {}
    fsm['convolutional_layer'] = {'before': ['convolutional_layer'],
                                  'add': ['convolutional_layer', 'maxpooling_layer']}
    fsm['maxpooling_layer'] = {'before': ['convolutional_layer'],
                               'add': ['convolutional_layer']}
    fsm['dense_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                          'add': ['dense_layer', 'dropout_layer']}
    fsm['dropout_layer'] = {'before': ['global_maxpooling_layer', 'dense_layer'],
                            'add': ['dense_layer']}

    return fsm

def addConvLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('num_filters' + str(idx), layerparameters['num_filters'][0],
                     layerparameters['num_filters'][1], layerparameters['num_filters'][2])
    toolboxes.append(toolbox.__getattribute__('num_filters' + str(idx)))
    toolbox.register('kernel_size' + str(idx), layerparameters['kernel_size'][0],
                     layerparameters['kernel_size'][1], layerparameters['kernel_size'][2])
    toolboxes.append(toolbox.__getattribute__('kernel_size' + str(idx)))
    toolbox.register('conv_activation_func' + str(idx), layerparameters['conv_activation_func'][0],
                     layerparameters['conv_activation_func'][1])
    toolboxes.append(toolbox.__getattribute__('conv_activation_func' + str(idx)))
    toolbox.register('conv_init_mode' + str(idx), layerparameters['conv_init_mode'][0],
                     layerparameters['conv_init_mode'][1])
    toolboxes.append(toolbox.__getattribute__('conv_init_mode' + str(idx)))
    toolbox.register('conv_weight_constraint' + str(idx), layerparameters['conv_weight_constraint'][0],
                     layerparameters['conv_weight_constraint'][1], layerparameters['conv_weight_constraint'][2])
    toolboxes.append(toolbox.__getattribute__('conv_weight_constraint' + str(idx)))

    defaultVal.update({'num_filters' + str(idx): 64})
    defaultVal.update({'kernel_size' + str(idx): 3})
    defaultVal.update({'conv_activation_func' + str(idx): "relu"})
    defaultVal.update({'conv_init_mode' + str(idx): "glorot_uniform"})
    defaultVal.update({'conv_weight_constraint' + str(idx): 3})


def addDenseLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('neurons' + str(idx), layerparameters['neurons'][0],
                     layerparameters['neurons'][1], layerparameters['neurons'][2])
    toolboxes.append(toolbox.__getattribute__('neurons' + str(idx)))
    toolbox.register('dense_activation_func' + str(idx), layerparameters['dense_activation_func'][0],
                     layerparameters['dense_activation_func'][1])
    toolboxes.append(toolbox.__getattribute__('dense_activation_func' + str(idx)))
    toolbox.register('dense_init_mode' + str(idx), layerparameters['dense_init_mode'][0],
                     layerparameters['dense_init_mode'][1])
    toolboxes.append(toolbox.__getattribute__('dense_init_mode' + str(idx)))
    toolbox.register('dense_weight_constraint' + str(idx), layerparameters['dense_weight_constraint'][0],
                     layerparameters['dense_weight_constraint'][1], layerparameters['dense_weight_constraint'][2])
    toolboxes.append(toolbox.__getattribute__('dense_weight_constraint' + str(idx)))

    defaultVal.update({'neurons' + str(idx): 1})
    defaultVal.update({'dense_activation_func' + str(idx): "relu"})
    defaultVal.update({'dense_init_mode' + str(idx): "glorot_uniform"})
    defaultVal.update({'dense_weight_constraint' + str(idx): 3})


def addMaxPoolingLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('pool_size' + str(idx), layerparameters['pool_size'][0],
                     layerparameters['pool_size'][1], layerparameters['pool_size'][2])
    toolboxes.append(toolbox.__getattribute__('pool_size' + str(idx)))

    defaultVal.update({'pool_size' + str(idx): 5})


def addDropoutLayer(idx, toolbox, toolboxes, defaultVal, layerparameters):
    toolbox.register('dropout_rate' + str(idx), layerparameters['dropout_rate'][0],
                     layerparameters['dropout_rate'][1], layerparameters['dropout_rate'][2])
    toolboxes.append(toolbox.__getattribute__('dropout_rate' + str(idx)))

    defaultVal.update({'dropout_rate' + str(idx): 0.2})

def getLayerSize(layer, conv_idx, dense_idx, dropout_idx, maxpooling_idx):
    if layer == 'convolutional_layer':
        conv_idx += 1
    elif layer == 'dense_layer':
        dense_idx += 1
    elif layer == 'dropout_layer':
        dropout_idx += 1
    elif layer == 'maxpooling_layer':
        maxpooling_idx += 1
    return conv_idx, dense_idx, dropout_idx, maxpooling_idx


def getMaxLayerSize(conv_idx, dense_idx, dropout_idx, maxpooling_idx, max_conv_idx, max_dense_idx, max_dropout_idx,
                    max_maxpooling_idx):
    if conv_idx > max_conv_idx:
        max_conv_idx = conv_idx
    if dense_idx > max_dense_idx:
        max_dense_idx = dense_idx
    if dropout_idx > max_dropout_idx:
        max_dropout_idx = dropout_idx
    if maxpooling_idx > max_maxpooling_idx:
        max_maxpooling_idx = maxpooling_idx

    return max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx


def addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, toolbox, toolboxes, defaultVal,
                      layerparameters):
    idx = 0
    while idx < max_conv_idx:
        idx += 1
        addConvLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)

    idx = 0
    while idx < max_dense_idx:
        idx += 1
        addDenseLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)

    idx = 0
    while idx < max_maxpooling_idx:
        idx += 1
        addMaxPoolingLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)

    idx = 0
    while idx < max_dropout_idx:
        idx += 1
        addDropoutLayer(idx, toolbox, toolboxes, defaultVal, layerparameters)


def generateFSM(n_pop, layerparameters, toolbox, toolboxes, defaultVal):
    fsm = FSM()

    path_ind = {}
    max_conv_idx = 0
    max_dense_idx = 0
    max_dropout_idx = 0
    max_maxpooling_idx = 0

    for ind in range(0, n_pop):
        idx = conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        path = [fsm[idx]['layer']]
        while len(fsm[idx]['next_path']) != 0:
            idx = random.choice(fsm[idx]['next_path'])
            layer = fsm[idx]['layer']
            path.append(layer)
            conv_idx, dense_idx, dropout_idx, maxpooling_idx = getLayerSize(layer, conv_idx, dense_idx, dropout_idx,
                                                                            maxpooling_idx)

        max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx = getMaxLayerSize(conv_idx, dense_idx,
                                                                                           dropout_idx, maxpooling_idx,
                                                                                           max_conv_idx, max_dense_idx,
                                                                                           max_dropout_idx,
                                                                                           max_maxpooling_idx)

        path_ind[ind] = path

    addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, toolbox, toolboxes, defaultVal,
                      layerparameters)

    return path_ind, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx


def openFSM(df, layerparameters, toolbox, toolboxes, defaultVal):
    path_ind = {}
    fitnesses = []

    hyperparams = [s for s in list(df.columns) if not 'Unnamed' in s]

    max_conv_idx = sum('num_filters' in s for s in hyperparams)
    max_dense_idx = sum('neurons' in s for s in hyperparams)
    max_dropout_idx = sum('dropout_rate' in s for s in hyperparams)
    max_maxpooling_idx = sum('pool_size' in s for s in hyperparams)

    for index, row in df.iterrows():
        path = [s for s in row if 'layer' in str(s)]
        fitness = [s for s in row if str(s).replace('.', '', 1).isdigit()]
        fitnesses.append(tuple([float(fitness[0])]))
        path_ind[index] = path

    addLayerToolboxes(max_conv_idx, max_dense_idx, max_dropout_idx, max_maxpooling_idx, toolbox, toolboxes, defaultVal,
                      layerparameters)

    return path_ind, fitnesses, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx

# CNN

In [4]:
import tensorflow as tf


class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=True))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters' + str(conv_idx)], indiv['kernel_size' + str(conv_idx)],
                                        kernel_initializer=indiv['conv_init_mode' + str(conv_idx)],
                                        activation=indiv['conv_activation_func' + str(conv_idx)],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint' + str(conv_idx)]),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons' + str(dense_idx)],
                                       kernel_initializer=indiv['dense_init_mode' + str(dense_idx)],
                                       activation=indiv['dense_activation_func' + str(dense_idx)],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint' + str(dense_idx)])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate' + str(dropout_idx)]))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size' + str(maxpooling_idx)]))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0,
                                 nesterov=False)
        elif indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                  decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                    decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                   schedule_decay=0.004)

        util = utility()
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[util.f1_m])

        return model

# Fitness Calculation

In [5]:
import collections
import os
from time import sleep

util = utility()
cnn = CNN()

def FitnessCalculation(individual, cfold, defaultVal, resultsPath, testing_name):
    indiv = collections.OrderedDict()
    i = 0
    for key in defaultVal.keys():
        indiv[key] = individual[i]
        i += 1

    path = individual[len(defaultVal):len(individual)]

    return crossfold(indiv, path, cfold, resultsPath, testing_name)


def crossfold(indiv, path, fold, resultsPath, testing_name):
    if indiv['output_dim'] == 50:
        embedding_mtx = fold['embedding_matrix'][0]
    elif indiv['output_dim'] == 100:
        embedding_mtx = fold['embedding_matrix'][1]
    elif indiv['output_dim'] == 200:
        embedding_mtx = fold['embedding_matrix'][2]
    elif indiv['output_dim'] == 300:
        embedding_mtx = fold['embedding_matrix'][3]

    model = cnn.cnn_model(fold['vocab_size'], fold['maxlen'], embedding_mtx,
                          indiv, path)
    
    #early stopping
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', mode='max', verbose=True, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint(resultsPath + testing_name + ".h5", monitor='val_f1_m', mode='max', verbose=False, 
                                  save_best_only=True)]

    class_weight = {0: 0.25,
                    1: 0.75}
    model.fit(fold['X_train'], fold['y_train'], epochs=indiv['epochs'], verbose=False, 
              validation_data=(fold['X_val'], fold['y_val']), use_multiprocessing=False,
              batch_size=indiv['batch_size'], callbacks=callbacks, class_weight=class_weight)
    
    dependencies = {
    'f1_m': util.f1_m
    }

    # load the saved model
    for x in range(0, 4):  # try 4 times
        try:
            # msg.send()
            saved_model = tf.keras.models.load_model(resultsPath + testing_name + ".h5", custom_objects=dependencies)
            str_error = None
        except Exception as e:
            print('An error occurs when loading saved model.')
            str_error = e
            pass

        if str_error:
            sleep(2)  # wait for 2 seconds before trying to fetch the data again
        else:
            break
    

    y_pred = saved_model.predict_classes(fold['X_val'])

    os.remove(resultsPath + testing_name + ".h5")

    # CNN metrics
    accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = util.get_testing_metric(fold['y_val'],
                                                                                                    y_pred)
    return f1ScoreBinary


# Genetic Algorithm

In [6]:
import random
from operator import attrgetter
from deap import base
from deap import creator
from deap import tools
import time
import datetime
import math
from scipy.spatial import distance
import itertools


class GeneticAlgorithm:
    __slots__ = (
        "toolbox", "toolboxes", "cross_rate", "mut_rate", "n_pop", "n_gen", "resultsPath", "testing_name", "cfold",
        "globalparameters", "layerparameters", "defaultVal", "path_ind", "max_conv_idx", "max_maxpooling_idx",
        "max_dense_idx", "max_dropout_idx")

    def __init__(self, toolbox, toolboxes, cross_rate, mut_rate, n_pop, n_gen, resultsPath, testing_name,
                 cfold, globalparameters, layerparameters, defaultVal, path_ind, max_conv_idx, max_maxpooling_idx,
                 max_dense_idx, max_dropout_idx):
        self.toolbox = toolbox
        self.toolboxes = toolboxes
        self.cross_rate = cross_rate
        self.mut_rate = mut_rate
        self.n_pop = n_pop
        self.n_gen = n_gen
        self.resultsPath = resultsPath
        self.testing_name = testing_name
        self.cfold = cfold
        self.globalparameters = globalparameters
        self.layerparameters = layerparameters
        self.defaultVal = defaultVal
        self.path_ind = path_ind
        self.max_conv_idx = max_conv_idx
        self.max_maxpooling_idx = max_maxpooling_idx
        self.max_dense_idx = max_dense_idx
        self.max_dropout_idx = max_dropout_idx

    def fitnessCalc(self, individual):
        i = 0
        if len(individual.fitness.values) == 0:
            if (0 in individual or '' in individual or 'False' in individual or None in individual):
                for param in self.defaultVal:
                    if individual[i] == 0 or individual[i] == '' or individual[i] == 'False' or individual[i] == None:
                        individual[i] = self.defaultVal[param]
                    i += 1

            fc = FitnessCalculation(individual, self.cfold, self.defaultVal, self.resultsPath, self.testing_name)
        else:
            fc = individual.fitness.values[0]
        print('{} {}'.format(datetime.datetime.now(), fc))
        return fc,

    def write_result(self):
        # Create Testing Results
        f = open(self.resultsPath + self.testing_name + ".csv", "a+")
        text = "i,min,max,mean,std,avgdistance,time,CR,MR"
        for param in self.defaultVal:
            text += ",{0}".format(param)
        text += "\n"       
        f.write(text)
        f.close()

        # Create Last Population file
        f = open(self.resultsPath + self.testing_name + "lastpop.csv", 'a+')
        text = "i,f1score"
        for param in self.defaultVal:
            text += ",{0}".format(param)
        text += "\n"
        f.write(text)
        f.close()

    def std_calc(self, fits, length):
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5

        return mean, std
    
    def distance_calc(self, pop):
        distances = []
        for subset in itertools.combinations(pop, 2):
            distances.append(distance.hamming(subset[0][0:subset[0].index('embedding_layer')],
                                              subset[1][0:subset[1].index('embedding_layer')]))

        avgDistance = sum(distances) / len(distances)
        
        return avgDistance

    def invalid_fitness_calc(self, pop):
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = map(self.toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

    def mutHyperparam(self, individual, indpb):
        toolboxesSize = len(self.toolboxes)
        fsm = FSM()
        mutatefsm = mutateFSM()
        addfsm = addFSM()

        # Mutation for the Hyperparameter Chromosomes
        for i in range(toolboxesSize):
            if random.random() < indpb:
                if len(self.toolboxes[i].args) == 1:
                    individual[i] = self.toolboxes[i].func(self.toolboxes[i].args[0])
                else:
                    individual[i] = self.toolboxes[i].func(self.toolboxes[i].args[0], self.toolboxes[i].args[1])

        # Mutation for the Architecture Chromosomes
        archChrom = individual[individual.index('convolutional_layer'):individual.index('output_layer')]
        size = len(archChrom)

        for i in range(1, size):
            if random.random() < indpb:
                if (i>=size):
                  break
                
                if (archChrom[i] == 'global_maxpooling_layer'):
                  continue

                selectMutType = random.randint(0, 2)
                # Remove the layer
                if selectMutType == 0:
                  for key in fsm:
                      if fsm[key]['layer'] in archChrom[i - 1:i]:
                          for j in fsm[key]['next_path']:
                              if i != size - 1 and i + 1 < size and fsm[j]['layer'] == archChrom[i + 1]:
                                  archChrom.remove(archChrom[i])
#                                   print('individual before remove', individual)
                                  individual[
                                  individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                   print('individual after remove', individual)
                                  size -= 1          
                                  break
                          else:
                            continue
                          break                              

                # Change the layer
                elif selectMutType == 1:
                    if i == size - 1:
                        if (archChrom[i] == 'dropout_layer') or (archChrom[i] == 'dense_layer' and 'dropout_layer' not in archChrom[i - 1:i]):
                            if (mutatefsm[archChrom[i]]['change'] == 'convolutional_layer' and individual.count(
                                    'convolutional_layer') < self.max_conv_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dense_layer' and individual.count('dense_layer')
                                    < self.max_dense_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'maxpooling_layer' and individual.count(
                                'maxpooling_layer') < self.max_maxpooling_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dropout_layer' and individual.count('dropout_layer')
                                    < self.max_dropout_idx):
                                archChrom[i] = mutatefsm[archChrom[i]]['change']
#                                 print('individual before change', individual)
                                individual[
                                individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                 print('individual after change', individual)

                    else:
                        if all(item in mutatefsm[archChrom[i]]['before'] for item in archChrom[i - 1:i]) and archChrom[i + 1] in mutatefsm[archChrom[i]]['after']:
                            if (mutatefsm[archChrom[i]]['change'] == 'convolutional_layer' and individual.count(
                                    'convolutional_layer') < self.max_conv_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dense_layer' and individual.count('dense_layer')
                                    < self.max_dense_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'maxpooling_layer' and individual.count(
                                'maxpooling_layer') < self.max_maxpooling_idx) or (
                                    mutatefsm[archChrom[i]]['change'] == 'dropout_layer' and individual.count('dropout_layer')
                                    < self.max_dropout_idx):
                                
                                archChrom[i] = mutatefsm[archChrom[i]]['change']
#                                 print('individual before change', individual)
                                individual[
                                individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                 print('individual after change', individual)
                # Add a layer
                elif selectMutType == 2:
                    for key in addfsm:
                        if key in archChrom[i] and all(item in addfsm[archChrom[i]]['before'] for item in archChrom[i - 1:i]):
                            if ('convolutional_layer' in addfsm[archChrom[i]]['add'] and individual.count(
                                    'convolutional_layer') < self.max_conv_idx) or (
                                    'dense_layer' in addfsm[archChrom[i]]['add'] and individual.count('dense_layer') < self.max_dense_idx) or (
                                    'maxpooling_layer' in addfsm[archChrom[i]]['add'] and individual.count(
                                'maxpooling_layer') < self.max_maxpooling_idx) or (
                                    'dropout_layer' in addfsm[archChrom[i]]['add'] and individual.count(
                                'dropout_layer') < self.max_dropout_idx):
                                archChrom.insert(i, random.choice(addfsm[archChrom[i]]['add']))
#                                 print('individual before add', individual)
                                individual[
                                individual.index('convolutional_layer'):individual.index('output_layer')] = archChrom
#                                 print('individual after add', individual)
                        
        return individual,

    def cxTwoPoint(self, ind1, ind2, pop, offspring):
        # Crossover for hyperparameter chromosomes
        size = ind1.index('embedding_layer')
        selectCxType = random.randint(0, 2)
        # One point crossover
        if selectCxType == 0:
#             print('ind1 before one-point crossover:', ind1)
#             print('ind2 before one-point crossover:', ind2)
            cxpoint = random.randint(1, size - 1)
            ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]
#             print('ind1 after one-point crossover:', ind1)
#             print('ind2 after one-point crossover:', ind2)
        # Two-point crossover
        elif selectCxType == 1:
#             print('ind1 before two-point crossover:', ind1)
#             print('ind2 before two-point crossover:', ind2)
            cxpoint1 = random.randint(1, size - 1)
            cxpoint2 = random.randint(1, size - 1)
            if cxpoint2 >= cxpoint1:
                cxpoint2 += 1
            else:  # Swap the two cx points
                cxpoint1, cxpoint2 = cxpoint2, cxpoint1

            ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
                = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]
#             print('ind1 after two-point crossover:', ind1)
#             print('ind2 after two-point crossover:', ind2)
        # Uniform crossover
        elif selectCxType == 2:
#             print('ind1 before uniform crossover:', ind1)
#             print('ind2 before uniform crossover:', ind2)
            for i in range(size):
                if random.random() < self.cross_rate:
                    ind1[i], ind2[i] = ind2[i], ind1[i]
#             print('ind1 after uniform crossover:', ind1)
#             print('ind2 after uniform crossover:', ind2)

        # Crossover for architecture chromosomes
        # One-cut point crossover from the Global MaxPooling layer
        cxpoint1 = ind1.index('global_maxpooling_layer')
        cxpoint2 = ind2.index('global_maxpooling_layer')
        ind1[cxpoint1:], ind2[cxpoint2:] = ind2[cxpoint2:], ind1[cxpoint1:]

        max_drop_layer = max(ind1.count('dropout_layer'), ind2.count('dropout_layer'))
        if max_drop_layer > self.max_dropout_idx:
            idx = self.max_dropout_idx
            self.max_dropout_idx = max_drop_layer

            while idx < self.max_dropout_idx:
                idx += 1
                addDropoutLayer(idx, self.toolbox, self.toolboxes, self.defaultVal, self.layerparameters)

                for ind in pop + offspring:
                    ind.insert(size, random.uniform(0, 1))
                size += 1

                self.write_result()

        return ind1, ind2

    def runGA(self, lastPop=[], lastFitnesses=[]):
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)

        self.toolbox.register("individual", tools.initCycle, creator.Individual,
                              self.toolboxes, n=1)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("evaluate", self.fitnessCalc)
        self.toolbox.register("mate", self.cxTwoPoint)
        self.toolbox.register("mutate", self.mutHyperparam, indpb=self.mut_rate)
        self.toolbox.register("select", tools.selBest)

        pop = self.toolbox.population(n=self.n_pop)

        idx = 0
        for ind in pop:
            if lastPop:
                ind[:] = lastPop[idx]
            ind.extend(self.path_ind[idx])
            idx += 1
        
        if lastFitnesses:
            # Fitnesses from previous population
            fitnesses = lastFitnesses
        else:
            # Evaluate the entire population
            fitnesses = list(map(self.toolbox.evaluate, pop))

        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        self.write_result()
        
        g = 0
        increment = 0.01
        avgDistance = 1
        threshold = 0.5
        minRate = 0.01
        maxRate = 1.0
        
        while g < self.n_gen:
            then = time.time()
            g = g + 1
            print('{} {}'.format(datetime.datetime.now(), "-- Generation %i --" % g))
            
            if g > 1:
                # Adaptive crossover and mutation rates
                print('Crossover rate before adaptive scheme=', self.cross_rate)
                print('Mutation rate before adaptive scheme=', self.mut_rate)   
                
                if avgDistance > threshold:
                    self.mut_rate -= increment
                    self.cross_rate -= increment
                else:
                    self.mut_rate += increment
                    self.cross_rate += increment
                    
                if self.mut_rate > maxRate:
                    self.mut_rate = maxRate
                elif self.mut_rate < minRate:
                    self.mut_rate = minRate

                if self.cross_rate > maxRate:
                    self.cross_rate = maxRate
                elif self.cross_rate < minRate:
                    self.cross_rate = minRate
                
                print('Crossover rate after adaptive scheme=', self.cross_rate)
                print('Mutation rate after adaptive scheme=', self.mut_rate)
            
            # Select the next generation individuals
            offspring = self.toolbox.select(pop, len(pop))
            # Clone the selected individuals
            offspring = list(map(self.toolbox.clone, offspring))

            # Apply crossover and mutation on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.cross_rate:
                    self.toolbox.mate(child1, child2, pop, offspring)
                    del child1.fitness.values
                    del child2.fitness.values

            for mutant in offspring:
                if random.random() < self.mut_rate:
                    self.toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Evaluate the individuals with an invalid fitness
            self.invalid_fitness_calc(offspring)

            pop[:] = self.toolbox.select(pop + offspring, self.n_pop)

            # Gather all the fitnesses in one list and print the stats
            fits = [ind.fitness.values[0] for ind in pop]

            length = len(pop)
            mean, std = self.std_calc(fits, length)
            avgDistance = self.distance_calc(pop)
            best = max(pop, key=attrgetter("fitness"))
            print('{} {}'.format(datetime.datetime.now(), "  Min %s" % min(fits)))
            print('{} {}'.format(datetime.datetime.now(), "  Max %s" % max(fits)))
            print('{} {}'.format(datetime.datetime.now(), "  Avg %s" % mean))
            print('{} {}'.format(datetime.datetime.now(), "  Std %s" % std))
            print('{} {}'.format(datetime.datetime.now(), "  AvgDistance %s" % avgDistance))
            print('{} {}'.format(datetime.datetime.now(), best))

            now = time.time()
            diff = now - then

            # save testing data
            f = open(self.resultsPath + self.testing_name + ".csv", 'a')
            text = "{0},{1},{2},{3},{4},{5},{6},{7},{8}".format(g,min(fits), max(fits), mean, std, avgDistance, diff, self.cross_rate, self.mut_rate)
            for param in best:
                text += ",{0}".format(param)
            text += "\n"
            f.write(text)
            f.close()

            # save last population data
            f = open(self.resultsPath + self.testing_name + "lastpop.csv", 'a')
            for ind in pop:
                text = "{0},{1}".format(g,ind.fitness.values[0])
                for param in ind:
                    text += ",{0}".format(param)
                text += "\n"                 
                f.write(text)

            f.close()            

# Project path

In [7]:
import os 
# path

training_path = 'TrainingData.csv'
trial_path = 'TrialData.csv'
population_path = 'NewPop.csv'
root_path = '/lab/dbms/fatyanosa'
datasetPath = root_path + '/Dataset/'
resultsPath = root_path + '/Server1/Results/'
archPath = root_path + '/Server1/Architecture/'
testing_name = "GA-CNN with Adaptive Balance CRMR"
glovePath = [root_path + '/Glove/glove.6B.50d.txt',
                 root_path + '/Glove/glove.6B.100d.txt',
                 root_path + '/Glove/glove.6B.200d.txt',
                 root_path + '/Glove/glove.6B.300d.txt']

# Parameters

In [8]:
# crossover rate is the probability with which two individuals
cross_rate = 0.8

# mutation rate is the probability for mutating an individual
mut_rate = 0.2

# number of population
n_pop = 30

# number of generation
n_gen = 100

# Main Program

In [9]:
import random
from sklearn.model_selection import StratifiedKFold
from deap import base
import warnings; warnings.simplefilter('ignore')

if __name__ == '__main__':
    globalparameters = []
    globalparameters.append(("epochs", random.randint, 1, 100))
    globalparameters.append(("batch_size", random.randint, 32, 256))

    globalparameters.append(("optimizer", random.choice, ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam',
                                                          'adamax', 'nadam']))
    globalparameters.append(("learning_rate", random.uniform, 1e-4, 1e-2))
    globalparameters.append(("momentum", random.uniform, 0, 1))
    globalparameters.append(("output_init_mode", random.choice,
                             ['zeros',
                              'ones',
                              'uniform',
                              'normal',
                              'glorot_normal',
                              'glorot_uniform',
                              'he_normal',
                              'he_uniform',
                              'lecun_normal',
                              'lecun_uniform']))
    globalparameters.append(("output_dim", random.choice, [50, 100, 200, 300]))

    layerparameters = {}
    layerparameters["num_filters"] = [random.randint, 32, 512]
    layerparameters["kernel_size"] = [random.randint, 1, 5]
    layerparameters["conv_activation_func"] = [random.choice,
                                               ['relu', 'softmax', 'elu', 'selu',
                                                'softplus', 'softsign', 'tanh',
                                                'sigmoid', 'hard_sigmoid', 'linear']]
    layerparameters["conv_init_mode"] = [random.choice,
                                         ['zeros',
                                          'ones',
                                          'uniform',
                                          'normal',
                                          'glorot_normal',
                                          'glorot_uniform',
                                          'he_normal',
                                          'he_uniform',
                                          'lecun_normal',
                                          'lecun_uniform']]
    layerparameters["conv_weight_constraint"] = [random.randint, 1, 5]
    layerparameters["neurons"] = [random.randint, 1, 30]
    layerparameters["dense_activation_func"] = [random.choice,
                                                ['relu', 'softmax', 'elu', 'selu',
                                                 'softplus', 'softsign', 'tanh',
                                                 'sigmoid', 'hard_sigmoid', 'linear']]
    layerparameters["dense_init_mode"] = [random.choice,
                                          ['zeros',
                                           'ones',
                                           'uniform',
                                           'normal',
                                           'glorot_normal',
                                           'glorot_uniform',
                                           'he_normal',
                                           'he_uniform',
                                           'lecun_normal',
                                           'lecun_uniform']]
    layerparameters["dense_weight_constraint"] = [random.randint, 1, 5]
    layerparameters["pool_size"] = [random.randint, 2, 6]
    layerparameters["dropout_rate"] = [random.uniform, 0, 1]

    defaultVal = collections.OrderedDict([
        ("epochs", 10),
        ("batch_size", 32),
        ("optimizer", "adam"),
        ("learning_rate", 1e-4),
        ("momentum", 0.9),
        ("output_init_mode", "glorot_uniform"),
        ("output_dim", 100)]
    )
    
    # object class
    util = utility()
    toolbox = base.Toolbox()
    toolboxes = []

    # Attribute generator
    for hyper in globalparameters:
        if len(hyper) == 3:
            toolbox.register(hyper[0], hyper[1], hyper[2])
        else:
            toolbox.register(hyper[0], hyper[1], hyper[2], hyper[3])

    toolboxes.append(toolbox.epochs)
    toolboxes.append(toolbox.batch_size)
    toolboxes.append(toolbox.optimizer)
    toolboxes.append(toolbox.learning_rate)
    toolboxes.append(toolbox.momentum)
    toolboxes.append(toolbox.output_init_mode)
    toolboxes.append(toolbox.output_dim)

    path_ind, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx = generateFSM(n_pop, layerparameters,
                                                                                             toolbox, toolboxes,
                                                                                             defaultVal)

#     # Read population data
#     dfPopulation = util.read_CSV(resultsPath + population_path)

#     path_ind, fitnesses, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx = openFSM(dfPopulation, layerparameters,
#                                                                                          toolbox, toolboxes, defaultVal)
#     dfPopulation = dfPopulation.drop(columns=[col for col in dfPopulation if col not in defaultVal])

#     population = dfPopulation.loc[:, ~dfPopulation.columns.str.match('Unnamed')].values.tolist()

    # Read data
    dfTraining = util.read_CSV(datasetPath + training_path)
    
    # Read trial data
    dfTrial = util.read_CSV(datasetPath + trial_path)

    textsTraining, labelsTraining = util.get_text_label(dfTraining)
    textsTrial, labelsTrial = util.get_text_label(dfTrial)
    cfold = {}

    X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix = util.get_training_trial_data(
        textsTraining, labelsTraining, textsTrial, labelsTrial, glovePath)
    cfold= {'X_train': X_train, 'X_val': X_val, 'y_train': y_train, 'y_val': y_val, 'vocab_size': vocab_size,
                  'maxlen': maxlen, 'embedding_matrix': embedding_matrix}
                  
    ga = GeneticAlgorithm(toolbox, toolboxes, cross_rate, mut_rate, n_pop, n_gen, resultsPath, testing_name,
                          cfold, globalparameters, layerparameters, defaultVal, path_ind, max_conv_idx, max_maxpooling_idx, max_dense_idx, max_dropout_idx)
    ga.runGA()
#     ga.runGA(population, fitnesses)


Epoch 00011: early stopping
2020-03-05 09:29:43.287555 0.6872928176795581
Epoch 00011: early stopping
2020-03-05 09:29:57.048968 0.6666666666666666
Epoch 00012: early stopping
2020-03-05 09:30:16.392657 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:30:51.604024 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:31:14.796528 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:31:34.002808 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:31:55.992700 0.0
Epoch 00011: early stopping
2020-03-05 09:32:12.897412 0.6666666666666666
2020-03-05 09:32:24.345355 0.6673286991062561
Epoch 00011: early stopping
2020-03-05 09:32:43.211069 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:32:56.058115 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:33:12.889561 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 09:33:38.232434 0.6666666666666666
Epoch 00014: early stopping
2020-03-05 09:33:54.981559 0.6666666666666666
Epoch 0

Epoch 00012: early stopping
2020-03-05 10:02:34.530140 0.723127035830619
Epoch 00017: early stopping
2020-03-05 10:02:59.164746 0.6772793053545587
Epoch 00011: early stopping
2020-03-05 10:03:22.055329 0.6666666666666666
Epoch 00013: early stopping
2020-03-05 10:03:47.305570 0.6666666666666666
Epoch 00011: early stopping
2020-03-05 10:03:55.195425 0.641202254226675
Epoch 00012: early stopping
2020-03-05 10:04:22.113762 0.6666666666666666
Epoch 00012: early stopping
2020-03-05 10:04:55.981248 0.7258172114743161
Epoch 00015: early stopping
2020-03-05 10:05:44.624284 0.6643258426966292
Epoch 00016: early stopping
2020-03-05 10:06:10.612604 0.6721390632544664
Epoch 00013: early stopping
2020-03-05 10:06:34.762274 0.7007454739084132
Epoch 00021: early stopping
2020-03-05 10:06:54.246995 0.6817010309278351
Epoch 00013: early stopping
2020-03-05 10:07:12.732650 0.672447013487476
Epoch 00013: early stopping
2020-03-05 10:07:34.826522 0.6883780332056195
Epoch 00013: early stopping
2020-03-05 10

Epoch 00011: early stopping
2020-03-05 10:31:01.168108 0.6817248459958932
Epoch 00012: early stopping
2020-03-05 10:31:26.941376 0.7064102564102565
2020-03-05 10:31:49.114379 0.6521739130434783
2020-03-05 10:32:09.152402 0.6962025316455698
2020-03-05 10:32:28.519558 0.6692197223551938
2020-03-05 10:33:00.499848 0.7048997772828508
2020-03-05 10:33:20.672001 0.7029634734665747
2020-03-05 10:33:41.722230 0.6757946210268949
Epoch 00011: early stopping
2020-03-05 10:34:11.984608 0.6901121304791029
Epoch 00013: early stopping
2020-03-05 10:34:34.551585 0.6752136752136754
Epoch 00016: early stopping
2020-03-05 10:35:00.380901 0.7084175084175085
Epoch 00025: early stopping
2020-03-05 10:35:16.300720 0.6533036377134372
Epoch 00013: early stopping
2020-03-05 10:35:38.664326 0.7009719839908518
Epoch 00011: early stopping
2020-03-05 10:35:59.331166 0.6755113322277501
Epoch 00016: early stopping
2020-03-05 10:36:23.849570 0.5962014863748968
Epoch 00016: early stopping
2020-03-05 10:36:51.107693 0.6

2020-03-05 10:59:57.386050 0.667065152420801
Epoch 00012: early stopping
2020-03-05 11:00:18.596699 0.6666666666666666
Epoch 00015: early stopping
2020-03-05 11:00:39.233105 0.6673049305887986
Epoch 00014: early stopping
2020-03-05 11:01:19.222663 0.6282335550628233
2020-03-05 11:01:39.329844 0.6721153846153847
2020-03-05 11:01:59.305060 0.580226904376013
2020-03-05 11:02:19.246064 0.6447368421052632
2020-03-05 11:02:39.303855 0.6685796269727404
2020-03-05 11:02:59.884686 0.6938483547925608
2020-03-05 11:03:20.232941 0.7048412285268089
2020-03-05 11:03:40.769349 0.6278409090909091
2020-03-05 11:04:08.413924 0.6705035971223021
2020-03-05 11:04:28.943123 0.7284991568296796
2020-03-05 11:04:49.216713 0.6688995215311005
2020-03-05 11:05:09.312207 0.6528704939919894
2020-03-05 11:05:29.361222 0.7050197405527354
Epoch 00012: early stopping
2020-03-05 11:05:58.651887 0.6821599453178401
Epoch 00023: early stopping
2020-03-05 11:06:35.961874 0.6704370179948586
Epoch 00016: early stopping
2020-0

2020-03-05 11:31:12.618140 0.6666666666666666
Epoch 00013: early stopping
2020-03-05 11:31:33.203722 0.6499608457321848
Epoch 00012: early stopping
2020-03-05 11:32:02.081930 0.6875912408759124
Epoch 00012: early stopping
2020-03-05 11:32:30.992475 0.6621417797888386
Epoch 00013: early stopping
2020-03-05 11:33:05.298680 0.6758409785932722
Epoch 00012: early stopping
2020-03-05 11:33:34.260296 0.7229064039408867
Epoch 00011: early stopping
2020-03-05 11:34:01.299752 0.6862257583291895
Epoch 00011: early stopping
2020-03-05 11:34:27.824124 0.7239927841250753
Epoch 00012: early stopping
2020-03-05 11:34:49.661094 0.6904145077720206
Epoch 00013: early stopping
2020-03-05 11:35:12.989661 0.7023498694516971
Epoch 00015: early stopping
2020-03-05 11:35:39.330204 0.6666666666666666
Epoch 00018: early stopping
2020-03-05 11:36:09.886840 0.6411201179071481
Epoch 00014: early stopping
2020-03-05 11:36:34.858184 0.691168963757019
Epoch 00012: early stopping
2020-03-05 11:36:56.577400 0.6999351911

...

Epoch 00013: early stopping
2020-03-06 02:44:13.070937 0.6630958757364755
2020-03-06 02:44:52.521137 0.725382262996942
2020-03-06 02:45:20.875424 0.7047828477185266
2020-03-06 02:45:49.721278 0.636623748211731
2020-03-06 02:46:19.113946 0.6717948717948717
2020-03-06 02:46:46.568494 0.7088888888888889
Epoch 00012: early stopping
2020-03-06 02:47:19.835018 0.6646058732612056
2020-03-06 02:47:52.197458 0.6666666666666666
2020-03-06 02:48:08.001096 0.6501007387508394
2020-03-06 02:48:36.127344 0.6987104337631886
2020-03-06 02:49:04.813899 0.6068759342301944
2020-03-06 02:49:33.658881 0.7186973610331274
2020-03-06 02:50:01.597199 0.6712243629583592
2020-03-06 02:50:30.241958 0.6666666666666666
2020-03-06 02:51:01.034088 0.683769116921559
2020-03-06 02:51:29.868891 0.6695760598503742
2020-03-06 02:51:58.679938 0.5563909774436091
2020-03-06 02:52:25.387830 0.6507324595219738
2020-03-06 02:52:49.829418 0.6750902527075812
Epoch 00013: early stopping
2020-03-06 02:53:29.813833 0.7089430894308945

2020-03-06 03:25:17.625530 0.6818727490996398
Epoch 00011: early stopping
2020-03-06 03:25:34.072137 0.0
2020-03-06 03:26:02.372365 0.6814516129032259
2020-03-06 03:27:12.346387 0.6280991735537191
2020-03-06 03:27:42.058448 0.6794598436389482
2020-03-06 03:28:11.356205 0.6860803727431567
2020-03-06 03:28:51.817242 0.6022813688212928
Epoch 00014: early stopping
2020-03-06 03:29:31.827085 0.6500366837857667
2020-03-06 03:30:00.225349 0.6500356379187456
2020-03-06 03:30:17.866366 0.6869695456814777
2020-03-06 03:30:46.958678 0.6657283603096411
2020-03-06 03:31:13.786477 0.6194029850746268
Epoch 00013: early stopping
2020-03-06 03:31:51.475971 0.6930440331844288
2020-03-06 03:32:22.122879 0.7257328990228012
Epoch 00011: early stopping
2020-03-06 03:32:59.437918 0.6666666666666666
2020-03-06 03:33:17.624797 0.7082748948106592
2020-03-06 03:33:47.623557 0.6706730769230769
2020-03-06 03:34:22.466232 0.5747316267547482
2020-03-06 03:34:50.539991 0.6841359773371104
2020-03-06 03:35:19.926596 0.

2020-03-06 04:09:13.316312 0.7325789160214413
2020-03-06 04:09:30.888128 0.6798780487804877
2020-03-06 04:09:49.700983 0.6768589298123697
2020-03-06 04:10:19.450155 0.022503516174402247
2020-03-06 04:10:37.263851 0.7354466858789627
Epoch 00011: early stopping
2020-03-06 04:11:09.745096 0.6772120612951064
Epoch 00011: early stopping
2020-03-06 04:11:29.711477 0.6850937808489636
2020-03-06 04:11:45.599134 0.6224256292906178
2020-03-06 04:12:15.559857 0.6997050147492626
2020-03-06 04:12:45.128299 0.6915750915750916
2020-03-06 04:13:26.909623 0.6718146718146718
Epoch 00016: early stopping
2020-03-06 04:14:11.318805 0.6216027874564459
2020-03-06 04:14:40.636681 0.7267338470657972
2020-03-06 04:15:04.910640 0.6757679180887373
2020-03-06 04:15:34.790822 0.6781292984869326
2020-03-06 04:16:48.356118 0.6666666666666666
2020-03-06 04:17:18.344051 0.7018984094407389
Epoch 00011: early stopping
2020-03-06 04:17:50.255168 0.667302192564347
2020-03-06 04:18:21.084338 0.7001338688085675
2020-03-06 04

2020-03-06 04:54:02.214652 0.0
2020-03-06 04:54:33.010133 0.6790281329923273
2020-03-06 04:55:04.633481 0.687603305785124
2020-03-06 04:55:24.400641 0.705758055995774
2020-03-06 04:55:52.475980 0.7153382451440053
2020-03-06 04:56:35.443172 0.5866666666666668
Epoch 00012: early stopping
2020-03-06 04:56:56.204037 0.6704980842911877
2020-03-06 04:57:39.486373 0.6609392898052692
2020-03-06 04:58:09.583045 0.6843156843156843
Epoch 00018: early stopping
2020-03-06 04:58:59.429505 0.6631158455392809
2020-03-06 04:59:17.665820 0.7346938775510203
2020-03-06 04:59:45.245559 0.7059447983014862
2020-03-06 05:00:16.453353 0.6826640548481882
2020-03-06 05:00:34.826887 0.6829268292682927
2020-03-06 05:00:51.053756 0.655128205128205
2020-03-06 05:01:28.417838 0.6842105263157895
Epoch 00014: early stopping
2020-03-06 05:02:09.340353 0.6996587030716723
2020-03-06 05:02:39.116549 0.6942637492607925
2020-03-06 05:03:09.613917 0.6845039018952063
2020-03-06 05:03:35.723499 0.6108786610878661
Epoch 00011: e

2020-03-06 05:39:15.461132 0.6579352850539291
2020-03-06 05:39:36.454667 0.7342368886269889
Epoch 00011: early stopping
2020-03-06 05:39:53.982721 0.6770276833414278
2020-03-06 05:40:12.983988 0.7197149643705463
2020-03-06 05:40:31.833524 0.6921985815602838
Epoch 00011: early stopping
2020-03-06 05:41:06.001437 0.0
Epoch 00011: early stopping
2020-03-06 05:41:32.149574 0.6743961352657004
Epoch 00012: early stopping
2020-03-06 05:41:53.692595 0.589937106918239
2020-03-06 05:42:22.027849 0.6444780635400907
2020-03-06 05:42:49.811006 0.7095081967213116
2020-03-06 05:43:15.381290 0.7022274325908558
2020-03-06 05:43:51.373655 0.6415929203539823
2020-03-06 05:44:24.363626 0.6803218727139722
2020-03-06 05:44:43.611958 0.6729559748427673
Epoch 00011: early stopping
2020-03-06 05:45:31.364952 0.7144563918757467
2020-03-06 05:46:03.809023 0.6685796269727404
2020-03-06 05:46:35.643913 0.6760290556900725
2020-03-06 05:47:07.644487 0.6987447698744771
2020-03-06 05:47:26.677303 0.665079365079365
202